# Estimating the probabilities of successive pairs

In [1]:
# Import all the usual things
import math
import random
import numpy as np

# Also stuff for reading files and processing strings
import string
import collections
from pathlib import Path 

# And, finally, stuff to write dictionaries out to files
# and read them back again.
import json

## Utilities to estimate the probabilities $p(a,b)$

Here I analyse a large body of training text to estimate the probability $p(a,b)$, which is the probability that symbol $a$ is followed by symbol $b$. As a symbol $a$ has to be followed by *something*, it follows that $\left( \sum_{b} p(a,b) \right) = 1$.

I'll estimate the $p(a,b)$ in a Bayesian way, using a calculation modelled on that in Section 5.2.1.7, entitled *Smoothing*, in
> S. Rogers & M. Girolami (2016), *A First Course in Machine Learning*, 2nd edition, Chapman & Hall/CRC. ISBN 9781498738484

A key feature of their approach is that yields a non-zero, though perhaps very small, probability for all pairs $ab$, even those that never appear in the traning text. This ameliorates a problem produced by the finiteness of the training data. Suppose that, in the training text, it never happens that an `n` is followed by a `w`. The simplest approach to estimating $p(n,w)$ would then yield $p(n,w) = 0$ and if we later tried to decode a message that contained the word `unwind`, we'd end up assigning a likelihood of zero to the correct cypher, purely becasue we'd underestimated $p(n,w)$.

From a technical point of view, we are going to do Bayesian inference for the $p(a,b)$ with a [multinomial](https://en.wikipedia.org/wiki/Multinomial_distribution) likelihood for the observed counts of successive letter pairs and a [Dirichlet](https://en.wikipedia.org/wiki/Dirichlet_distribution) prior on the probabilities. Because these distributions are [conjugate](https://en.wikipedia.org/wiki/Conjugate_prior), we'll end up with a Dirichlet posterior as well, and we'll then make a maximum-*a-posteriori* (MAP) estimate for the $p(a,b)$.

### Standardising the text

Define a function to standardise the text by making it all lowercase and replacing characters that aren't in the allowed alphabet with blank spaces.

In [2]:
def standardiseText( rawText, allowedChars, replacementChar=' ' ):
    # Make all the characters lower case
    rawText = rawText.lower()

    # Replace any characters that aren't part of our list
    # of allowed characters with the replacement character
    standardisedText = ""
    for char in rawText:
        if allowedChars.find(char) == -1:
            # char isn't one of the allowed ones
            standardisedText = standardisedText + replacementChar
        else:
            standardisedText = standardisedText + char
            
    return( standardisedText )

# Do a small test
testText = "Where would heavy metal be without the ümlaut?"
standardiseText( testText, ' abcdefghijklmnopqrstuvwxyz', '*' )

'where would heavy metal be without the *mlaut*'

### The main event: slurp the training text, count pairs and estimate $p(a,b)$.

This function does the bulk of the work of estimating the probabilities.

In [3]:
def findLogPairProbs( trainingTextFiles, allowedChars ):
    ############################################
    # Read and standardise the training texts
    ############################################
    # Read the whole training text into a single string 
    # See https://stackoverflow.com/questions/1631897/python-file-slurp
    rawTrainingText = ''
    for textFile in trainingTextFiles:
        rawTrainingText += Path(textFile).read_text()
        
    print( 'The training text is ' + str(len(rawTrainingText)) + ' characters long.')
    
    ###########################################
    # Standardise the training text
    ###########################################
    # Make sure the list of allowed characters
    # is sorted into a standard order and doesn't 
    # contain any repeats.
    charSet = sorted( list( set(allowedChars) ) ) 
    charSetStr = ''.join(charSet)
    nSymbols = len(charSetStr)
    
    # Standardise the text
    stdTrainingText = standardiseText( rawTrainingText, charSetStr, ' ' )        
    
    ##########################################
    # Count appearances of pairs
    ##########################################
    # Build a dictionary whose keys are the allowed characters
    # and whose values are integers. They'll eventually be counts.
    emptyCountDict = dict.fromkeys( charSet, 0 )

    # Build a dictionary whose keys are the allowed characters 
    # and whose values are copies of the empty count dictionary
    pairCounts = dict.fromkeys( charSet )
    for char in pairCounts.keys():
        pairCounts[char] = emptyCountDict.copy()

    # Now count appearances of pairs
    for j in range(1, len(stdTrainingText)):
        firstChar = stdTrainingText[j-1]
        secondChar = stdTrainingText[j]
        pairCounts[firstChar][secondChar] += 1
        
    ###########################################
    # Estimate the probabilities, or rather, 
    # their logs
    ###########################################
    # Build a dictionary-of-dictionaries that holds
    # the parameters of the Dirichlet posteriors.
    priorAlpha = 2 # favours broadly uniform, nonzero probabilities for all letter pairs

    dirichletPosterior = dict.fromkeys( charSet )
    for firstChar in dirichletPosterior.keys():
        dirichletPosterior[firstChar] = dict.fromkeys( charSet )
        for secondChar in dirichletPosterior[firstChar].keys():
            dirichletPosterior[firstChar][secondChar] = pairCounts[firstChar][secondChar] + priorAlpha
            
    # Get the MAP estimates for the probabilities
    logPairProbs = dict.fromkeys( charSet )
    for firstChar in logPairProbs.keys():
        # Add up all the entries in the posterior for this row
        posteriorAlphaSum = 0.0
        for secondChar in dirichletPosterior[firstChar].keys():
            posteriorAlphaSum += dirichletPosterior[firstChar][secondChar]

        # Initialise the result
        logPairProbs[firstChar] = dict.fromkeys( charSet, 0.0 )

        # Get the logs of the maximum-a-posteriori estimates por the probs
        logNomalisation = math.log( posteriorAlphaSum - len(dirichletPosterior[firstChar].keys()) )
        for secondChar in logPairProbs[firstChar].keys():
            logPairProbs[firstChar][secondChar] = math.log(dirichletPosterior[firstChar][secondChar] - 1) - logNomalisation
    
    return( logPairProbs )

## Applying the tools and estimating $p(a,b)$

Here we apply the tools of the previous section.

In [4]:
trainingTextFiles = ['CaoJoly_DreamOfTheRedChamber_Vol1.txt', 'CaoJoly_DreamOfTheRedChamber_Vol2.txt']
commonChars = "abcdefghijklmnopqrstuvwxyz0123456789 ,.?!:;"
logPairProbs = findLogPairProbs( trainingTextFiles, commonChars )

The training text is 2505573 characters long.


As a check, assemble the results into a matrix and check that the rows sum to one.

In [5]:
nSymbols = len(logPairProbs.keys())
probMat = np.zeros( (nSymbols, nSymbols) )

row = 0
for firstChar in logPairProbs.keys():
    probMat[row,:] = np.exp( np.array(list(logPairProbs[firstChar].values())))
    row += 1
    
np.sum( probMat, axis=1 ) # sums across rows

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1.])

#### Write the logs of the $p(a,b)$ to a file.

Here we write the log dictionary out and then read it back.

In [6]:
# Write the dictionary out to a file
pairProbFile = "LogPairProbDict.json"
jsonForProbDict = json.dumps(logPairProbs)
myFileObj = open(pairProbFile, "w")
myFileObj.write(jsonForProbDict)
myFileObj.close()

In [7]:
# Read it back 
myFileObj = open(pairProbFile, "r")
jsonFromFile = myFileObj.read()
dictFromFile = json.loads(jsonFromFile)
myFileObj.close()

# Check whether the two versions agree
dictFromFile == logPairProbs

True